In [ ]:
#TODO: reduce size vocabulary
#TODO: check size embedding
#TODO: tensorboard

In [3]:
import pandas as pd

df = pd.read_csv('data/crowdflower_data.csv')

In [5]:
df.head()

,sentiment,content
0,sadness,Layin n bed with a headache ughhhh...waitin o...
1,sadness,Funeral ceremony...gloomy friday...
2,neutral,@dannycastillo We want to trade with someone w...
3,worry,Re-pinging @ghostridah14: why didn't you go to...
4,sadness,"I should be sleep, but im not! thinking about ..."


In [6]:
df['sentiment'].unique()

array(['sadness', 'neutral', 'worry', 'happiness'], dtype=object)

In [7]:
df.head()

,sentiment,content
0,sadness,Layin n bed with a headache ughhhh...waitin o...
1,sadness,Funeral ceremony...gloomy friday...
2,neutral,@dannycastillo We want to trade with someone w...
3,worry,Re-pinging @ghostridah14: why didn't you go to...
4,sadness,"I should be sleep, but im not! thinking about ..."


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])
sequences = pad_sequences(sequences, maxlen=100)

In [10]:
import numpy as np

emotion_dict = {}
labels = df['sentiment']

for k in range(len(labels.unique())):
    emotion_dict[labels.unique()[k]] = k
    
encoded_labels = np.array(list(labels.map(lambda x: emotion_dict[x])))

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences, encoded_labels, stratify=encoded_labels)

In [12]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).shuffle(1024).batch(32)

In [13]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

model = tf.keras.Sequential()

model.add(Embedding(len(tokenizer.word_index) +1, 50))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, dropout=0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(len(df['sentiment'].unique()), activation='softmax'))

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          1866950   
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (None, None, 50)          20200     
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 32)                1632      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
Total p

In [14]:
model.fit(X_train, y_train, epochs = 10, validation_data=(X_test,y_test))

Train on 20603 samples, validate on 6868 samples
Epoch 1/10
20603/20603 [==============================] - 136s 7ms/sample - loss: 1.2638 - accuracy: 0.4113 - val_loss: 1.1665 - val_accuracy: 0.4712
Epoch 2/10
20603/20603 [==============================] - 92s 4ms/sample - loss: 1.0486 - accuracy: 0.5449 - val_loss: 1.1923 - val_accuracy: 0.4632
Epoch 3/10
20603/20603 [==============================] - 97s 5ms/sample - loss: 0.8026 - accuracy: 0.6768 - val_loss: 1.3273 - val_accuracy: 0.4471
Epoch 4/10
 9760/20603 [=============>................] - ETA: 45s - loss: 0.5407 - accuracy: 0.8061

KeyboardInterrupt: 